In [109]:
import pandas as pd
import numpy as np

In [110]:
print("Hello World")

Hello World


In [111]:
# At first imported all the required data.
df_summary = pd.read_csv("dim_match_summary.csv")
df_players = pd.read_csv("dim_players.csv")
df_batting = pd.read_csv("fact_bating_summary.csv")
df_bowling = pd.read_csv("fact_bowling_summary.csv")

#Primary Insights

1. Top 10 Batsman based on past 3 years total runs scored.

In [112]:
top_10_batsman_based_on_runs = df_batting.groupby(['batsmanName'])['runs'].sum().sort_values(ascending=False).head(10)

top_10_batsman_based_on_runs

batsmanName
ShubmanGill        1851
FafduPlessis       1831
RuturajGaikwad     1593
KLRahul            1516
JosButtler         1509
ShikharDhawan      1392
ViratKohli         1385
SanjuSamson        1304
SuryakumarYadav    1225
GlennMaxwell       1214
Name: runs, dtype: int64

2. Top 10 Batsmen based on past 3 years batting average.(min 60 balls faced in each season)

In [113]:
# STEP:-1 first we need to fetch the year data from the matchDate column which is in df_summary.
df_summary['season'] = df_summary['matchDate'].apply(lambda x : x.split(",")[-1])
df_summary

,team1,team2,winner,margin,matchDate,match_id,season
0,Super Kings,KKR,Super Kings,27 runs,"Oct 15, 2021",T203817,2021
1,Capitals,KKR,KKR,3 wickets,"Oct 13, 2021",T206442,2021
2,RCB,KKR,KKR,4 wickets,"Oct 11, 2021",T208597,2021
3,Capitals,Super Kings,Super Kings,4 wickets,"Oct 10, 2021",T201819,2021
4,Capitals,RCB,RCB,7 wickets,"Oct 8, 2021",T205942,2021
...,...,...,...,...,...,...,...
201,Mumbai,RCB,RCB,8 wickets,"Apr 2, 2023",T202485,2023
202,Royals,Sunrisers,Royals,72 runs,"Apr 2, 2023",T209117,2023
203,Super Giants,Capitals,Super Giants,50 runs,"Apr 1, 2023",T208512,2023
204,Punjab Kings,KKR,Punjab Kings,7 runs,"Apr 1, 2023",T202853,2023


In [114]:
# This line of code creates a new column named "out" in the DataFrame df_batting.
# It assigns a value of 1 to each row where the value 'out', indicating that the batsman got out, and a value of 0 otherwise.

df_batting["out"] = df_batting['out/not_out'].apply(lambda x : 1 if x == 'out' else 0)

# It creates a new DataFrame named new_df containing, merges two DataFrames, df_batting and df_summary, based on the common column 'match_id'.
# It specifically selects only the 'match_id' and 'season' columns from df_summary DataFrame to merge.
new_df = df_batting.merge(df_summary[['match_id','season']],on='match_id')

In [115]:
# This line of code calculates the cumulative sum of balls faced, runs scored, and times out for each batsman in each season and stores the result 
# in a DataFrame named temp_1.
temp_1 = new_df.groupby(['batsmanName','season']).agg({'balls':'sum','runs':'sum','out':'sum'}).reset_index()


# This line of code filters the DataFrame temp_1 to include only those rows where each batsman has faced at least 60 balls, and stored in a 
# DataFrame named min_60_balls.
min_60_balls = temp_1.groupby('batsmanName').filter(lambda x : (x['balls']>=60).all())
min_60_balls

,batsmanName,season,balls,runs,out
0,ABdeVilliers,2021,211,313,10
1,AaronFinch,2022,61,86,5
7,AbhinavManohar,2022,75,108,6
8,AbhinavManohar,2023,83,114,6
9,AbhishekSharma,2021,69,93,5
...,...,...,...,...,...
491,WriddhimanSaha,2022,259,317,10
492,WriddhimanSaha,2023,287,371,16
495,YashasviJaiswal,2021,168,249,10
496,YashasviJaiswal,2022,194,258,10


In [116]:
# This code calculates the names of batsmen who have played in exactly three seasons based on the provided DataFrame.

temp_2 = new_df.groupby('batsmanName')['season'].nunique().reset_index()
filtered_batsman = temp_2[temp_2['season'] == 3]['batsmanName'].values

In [117]:
# This code filters the DataFrame min_60_balls to include only the rows where the 'batsmanName' column matches any of the names in the
# filtered_batsman array, which contains the names of batsmen who have played in exactly three seasons.

filtered_data = min_60_balls[min_60_balls["batsmanName"].isin(filtered_batsman)]
filtered_data

,batsmanName,season,balls,runs,out
9,AbhishekSharma,2021,69,93,5
10,AbhishekSharma,2022,320,426,14
11,AbhishekSharma,2023,157,226,11
16,AidenMarkram,2021,119,146,5
17,AidenMarkram,2022,274,381,8
...,...,...,...,...,...
491,WriddhimanSaha,2022,259,317,10
492,WriddhimanSaha,2023,287,371,16
495,YashasviJaiswal,2021,168,249,10
496,YashasviJaiswal,2022,194,258,10


In [118]:
# This code calculates the batting average for each batsman in the filtered data by dividing the total runs scored by the total number of times 
# they got out, and rounding the result to two decimal places. Then, it selects the top 10 batsmen with the highest batting averages.

top_10_batsman_on_batting_avg = round((filtered_data.groupby('batsmanName')['runs'].sum()/filtered_data.groupby('batsmanName')['out'].sum()),2)
top_10_batsman_on_batting_avg = top_10_batsman_on_batting_avg.nlargest(10)

top_10_batsman_on_batting_avg

batsmanName
KLRahul            50.53
FafduPlessis       43.60
DavidMiller        43.20
JosButtler         41.92
ShimronHetmyer     40.67
ShubmanGill        40.24
ShikharDhawan      39.77
RuturajGaikwad     37.93
DavidWarner        37.90
SuryakumarYadav    35.00
dtype: float64

3. Top 10 batsmen based on past 3 years strike rate (min 60 balls faced in each season)

In [119]:
# # This code calculates the strike rate for each batsman in the filtered data(which we created earlier) by dividing the total runs scored by the 
# total number of balls faced, then multiplying by 100 to convert it to a percentage. Afterwards, it rounds the result to two decimal places and 
# selects the top 10 batsmen with the highest strike rates.

top_batsman_strikeRate = (filtered_data.groupby('batsmanName')['runs'].sum()/filtered_data.groupby('batsmanName')['balls'].sum())*100
top_batsman_on_strikeRate = round(top_batsman_strikeRate.nlargest(10),2)

In [120]:
top_batsman_on_strikeRate

batsmanName
GlennMaxwell       161.44
SuryakumarYadav    160.55
AndreRussell       159.19
ShimronHetmyer     157.27
NicholasPooran     157.11
PrithviShaw        153.20
DineshKarthik      152.64
YashasviJaiswal    152.15
JosButtler         146.93
ShivamDube         145.95
dtype: float64

4. Top 10 bowlers based on past 3 years total wickets taken.

In [121]:
# This code calculates the total number of wickets taken by each bowler in the DataFrame df_bowling, then sorts the result in descending order and 
# selects the top 10 bowlers with the highest number of wickets.

top_10_bowlers = df_bowling.groupby('bowlerName')['wickets'].sum().sort_values(ascending=False).head(10)
top_10_bowlers

bowlerName
MohammedShami        67
YuzvendraChahal      66
HarshalPatel         65
RashidKhan           63
AveshKhan            47
KagisoRabada         45
ArshdeepSingh        45
VarunChakravarthy    44
ShardulThakur        43
TrentBoult           42
Name: wickets, dtype: int64

5. Top 10 bowlers based on past 3 years bowling average. (min 60 balls bowled in each season).

In [122]:
# This code merges the DataFrame df_bowling with selected columns from df_summary (specifically 'match_id' and 'season'), using the 'match_id' column 
# as the key. The resulting DataFrame bowling_data will contain information about bowling performances along with the corresponding match's season.

bowling_data = df_bowling.merge(df_summary[['match_id','season']],on='match_id')
bowling_data

,match_id,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,season
0,T203817,Super Kings Vs KKR,KKR,ShakibAlHasan,3.0,0,33,0,11.00,6,2,3,0,0,2021
1,T203817,Super Kings Vs KKR,KKR,ShivamMavi,4.0,0,32,1,8.00,8,1,2,0,0,2021
2,T203817,Super Kings Vs KKR,KKR,LockieFerguson,4.0,0,56,0,14.00,4,7,2,2,0,2021
3,T203817,Super Kings Vs KKR,KKR,VarunChakravarthy,4.0,0,38,0,9.50,5,2,2,0,1,2021
4,T203817,Super Kings Vs KKR,KKR,SunilNarine,4.0,0,26,2,6.50,6,0,1,1,0,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2431,T201566,Super Kings Vs Titans,Super Kings,DeepakChahar,4.0,0,29,0,7.25,9,1,2,0,0,2023
2432,T201566,Super Kings Vs Titans,Super Kings,TusharDeshpande,3.2,0,51,1,15.30,5,4,4,1,1,2023
2433,T201566,Super Kings Vs Titans,Super Kings,RajvardhanHangargekar,4.0,0,36,3,9.00,10,4,1,3,1,2023
2434,T201566,Super Kings Vs Titans,Super Kings,MitchellSantner,4.0,0,32,0,8.00,7,5,0,0,0,2023


In [123]:
# This code calculates the total number of balls bowled by each player by multiplying the number of overs bowled by 6 (since each over consists 
# of 6 balls) and assigns the result to a new column 'balls' in the DataFrame bowling_data. 

bowling_data['balls'] = bowling_data['overs']*6
bowling_data

,match_id,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,season,balls
0,T203817,Super Kings Vs KKR,KKR,ShakibAlHasan,3.0,0,33,0,11.00,6,2,3,0,0,2021,18.0
1,T203817,Super Kings Vs KKR,KKR,ShivamMavi,4.0,0,32,1,8.00,8,1,2,0,0,2021,24.0
2,T203817,Super Kings Vs KKR,KKR,LockieFerguson,4.0,0,56,0,14.00,4,7,2,2,0,2021,24.0
3,T203817,Super Kings Vs KKR,KKR,VarunChakravarthy,4.0,0,38,0,9.50,5,2,2,0,1,2021,24.0
4,T203817,Super Kings Vs KKR,KKR,SunilNarine,4.0,0,26,2,6.50,6,0,1,1,0,2021,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2431,T201566,Super Kings Vs Titans,Super Kings,DeepakChahar,4.0,0,29,0,7.25,9,1,2,0,0,2023,24.0
2432,T201566,Super Kings Vs Titans,Super Kings,TusharDeshpande,3.2,0,51,1,15.30,5,4,4,1,1,2023,19.2
2433,T201566,Super Kings Vs Titans,Super Kings,RajvardhanHangargekar,4.0,0,36,3,9.00,10,4,1,3,1,2023,24.0
2434,T201566,Super Kings Vs Titans,Super Kings,MitchellSantner,4.0,0,32,0,8.00,7,5,0,0,0,2023,24.0


In [124]:
# This code first aggregates bowling statistics by combining data for each bowler across seasons. It sums up runs conceded, balls bowled, wickets taken, 
# and overs bowled for each bowler in each season. Then, it resets the index to convert the result into a DataFrame.

# After that, it filters the aggregated data to select only those bowlers who have bowled a minimum of 60 balls across all seasons.

temp_3 = bowling_data.groupby(['bowlerName','season']).agg({'runs':'sum','balls':'sum','wickets':'sum','overs':'sum'}).reset_index()

filtered_bowlers = temp_3.groupby('bowlerName').filter(lambda x : (x['balls']>=60).all())

In [125]:
# This code calculates the number of seasons each bowler has played based on the aggregated bowling data in temp_3. Then, it filters the data to 
# select only those bowlers who have played in exactly three seasons. 

temp_4 = temp_3.groupby('bowlerName')['season'].nunique().reset_index()
bowlers_name = temp_4[temp_4['season']==3]['bowlerName'].values

In [126]:
# This code filters the DataFrame filtered_bowlers to include only the rows where the 'bowlerName' column matches any of the names in the bowlers_name 
# array, which contains the names of bowlers who have played in exactly three seasons. The resulting DataFrame is stored in filtered_bowler_data.

filtered_bowler_data = filtered_bowlers[filtered_bowlers['bowlerName'].isin(bowlers_name)]
filtered_bowler_data

,bowlerName,season,runs,balls,wickets,overs
24,AndreRussell,2021,188,114.0,11,19.0
25,AndreRussell,2022,278,166.2,17,27.7
26,AndreRussell,2023,172,90.6,7,15.1
28,AnrichNortje,2021,187,181.2,12,30.2
29,AnrichNortje,2022,217,133.2,9,22.2
...,...,...,...,...,...,...
360,WashingtonSundar,2022,239,168.0,6,28.0
361,WashingtonSundar,2023,146,104.4,3,17.4
368,YuzvendraChahal,2021,374,318.0,18,53.0
369,YuzvendraChahal,2022,527,408.0,27,68.0


In [127]:
# This code calculates the bowling average for each bowler in the filtered bowling data by dividing the total runs conceded by the total number 
# of wickets taken, and rounding the result to two decimal places. Then, it selects the top 10 bowlers with the lowest bowling averages.

top_bowlers_bowling_avg = round((filtered_bowler_data.groupby('bowlerName')['runs'].sum()/filtered_bowler_data.groupby('bowlerName')['wickets'].sum()),2)
top_bowlers_bowling_avg.nlargest(10)

bowlerName
ChrisJordan           52.00
SandeepSharma         50.13
WashingtonSundar      41.92
LalitYadav            36.90
RavichandranAshwin    35.61
KrunalPandya          35.12
BhuvneshwarKumar      33.62
RileyMeredith         32.21
MohammedSiraj         31.85
AxarPatel             31.30
dtype: float64

6. Top 10 bowlers based on past 3 years economy rate. (min 60 balls bowled in each season)

In [128]:
# This code calculates the economy rate for each bowler in the filtered bowling data by dividing the total runs conceded by the total number of 
# overs bowled. Then, it selects the top 10 bowlers with the lowest economy rates.

top_bowlers_economy_rate = (filtered_bowler_data.groupby('bowlerName')['runs'].sum()/filtered_bowler_data.groupby('bowlerName')['overs'].sum())
top_bowlers_economy_rate = top_bowlers_economy_rate.nsmallest(10)

In [129]:
top_bowlers_economy_rate

bowlerName
SunilNarine           6.600000
MoeenAli              7.086280
AxarPatel             7.113636
RashidKhan            7.216438
KrunalPandya          7.453581
RavindraJadeja        7.455882
RavichandranAshwin    7.532051
VarunChakravarthy     7.584693
HarpreetBrar          7.609682
RaviBishnoi           7.676408
dtype: float64

7. Top 5 batsmen based on past 3 years boundary % (fours and sixes).

In [130]:
# This code adds a new column 'boundaries' to the DataFrame df_batting, which contains the sum of '4s' (fours) and '6s' (sixes) hit by each batsman. 
# Then, it calculates the total number of balls faced by each batsman and stores the result in the DataFrame temp. Finally, it filters out batsmen 
# who have faced more than 150 balls and stores their names in the array batsman.

df_batting['boundries'] = df_batting['4s'] + df_batting['6s']
temp = df_batting.groupby('batsmanName')['balls'].sum().reset_index()
batsman = temp[temp['balls'] > 150]['batsmanName'].values

In [131]:
# This code filters the DataFrame df_batting to include only the rows where the 'batsmanName' column matches any of the names in the batsman array,
# which contains the names of batsmen who have faced more than 150 balls

boundaries_data = df_batting[df_batting['batsmanName'].isin(batsman)]
boundaries_data

,match_id,match,teamInnings,battingPos,batsmanName,out/not_out,runs,balls,4s,6s,SR,out,boundries
0,T203817,Super Kings Vs KKR,Super Kings,1,RuturajGaikwad,out,32,27,3,1,118.51,1,4
1,T203817,Super Kings Vs KKR,Super Kings,2,FafduPlessis,out,86,59,7,3,145.76,1,10
2,T203817,Super Kings Vs KKR,Super Kings,3,RobinUthappa,out,31,15,0,3,206.66,1,3
3,T203817,Super Kings Vs KKR,Super Kings,4,MoeenAli,not_out,37,20,2,3,185.00,0,5
4,T203817,Super Kings Vs KKR,KKR,1,ShubmanGill,out,51,43,6,0,118.60,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3263,T201566,Super Kings Vs Titans,Titans,3,SaiSudharsan,out,22,17,3,0,129.41,1,3
3264,T201566,Super Kings Vs Titans,Titans,4,HardikPandya,out,8,11,0,0,72.72,1,0
3265,T201566,Super Kings Vs Titans,Titans,5,VijayShankar,out,27,21,2,1,128.57,1,3
3266,T201566,Super Kings Vs Titans,Titans,6,RahulTewatia,not_out,15,14,1,1,107.14,0,2


In [132]:
# This code calculates the percentage of runs scored through boundaries (fours and sixes) for each batsman in the filtered batting data by dividing 
# the total number of boundaries hit by the total number of balls faced, then multiplying by 100 to convert it to a percentage. 
# Finally, selects the top 5 batsmen with the highest percentage of runs scored through boundaries.

top_batsman_on_boundaries = (boundaries_data.groupby('batsmanName')['boundries'].sum()/boundaries_data.groupby('batsmanName')['balls'].sum())*100
top_batsman_on_boundaries.nlargest(5)

batsmanName
RashidKhan         25.581395
YashasviJaiswal    25.268817
PrithviShaw        24.436090
SuryakumarYadav    23.853211
KyleMayers         23.577236
dtype: float64

8. Top 5 bowlers based on past 3 years dot ball %.

In [133]:
# This code calculates the total number of balls bowled by each player in the DataFrame df_bowling. It multiplies the number of overs bowled 
# ('overs' column) by 6 (since each over consists of 6 balls) and assigns the result to a new column 'balls'

df_bowling['balls'] = df_bowling['overs']*6

In [134]:
# This code calculates the percentage of dot balls bowled by each bowler in the DataFrame df_bowling. It divides the total number of dot balls 
# ('0s' column) by the total number of balls bowled, then multiplies by 100 to convert it to a percentage.

# Finally,selects the top 5 bowlers with the highest percentage of dot balls.

top_5_bowlers = (((df_bowling.groupby('bowlerName')['0s'].sum())/df_bowling.groupby('bowlerName')['balls'].sum())*100)

top_5_bowler_with_highest_dotballs_percentage = round(top_5_bowlers.sort_values(ascending=False),2)
top_5_bowler_with_highest_dotballs_percentage.head(5)

bowlerName
ShreyasIyer       66.67
ReeceTopley       58.33
ImranTahir        58.33
DewaldBrevis      55.56
SimarjeetSingh    52.78
dtype: float64

9. Top 4 teams based on past 3 years winning %.

In [135]:
# This code calculates the winning percentage for each team based on the number of matches won by each team divided by the total number of 
# matches played, then multiplied by 100 to convert it to a percentage and selects the top 4 teams with the highest winning percentages.

winning_percentage = (df_summary['winner'].value_counts()/df_summary['match_id'].nunique())*100
winning_percentage.sort_values(ascending=False).nlargest(4)

winner
Super Kings    12.135922
RCB            12.135922
Titans         11.165049
Royals         10.679612
Name: count, dtype: float64

10.Top 2 teams with the highest number of wins achieved by chasing targets over the past 3 years.

In [57]:
chasing_team = df_batting[['match_id','match','teamInnings']].merge(df_summary,on='match_id',how='inner')

temp = chasing_team[(chasing_team['teamInnings'] == chasing_team['team2'])]
temp

chasing_winning_team = temp[temp['winner'] == temp['team2']]['winner'].value_counts().nlargest(2)

chasing_winning_team

winner
KKR         93
Capitals    84
Name: count, dtype: int64

#Secondary Sights

Secondary Insights (This will require additional research)
Predict the following for the season 2024 using available data and by doing additional
research
1. Orange and purple cap player
2. Top 4 qualifying teams
3. Winner and runner-up
Your Picks
4. Pick your team selecting the Best 11 players based on their positions, 3 years
performance data and additional research
5. Pick your top 3 all-rounders

In [666]:
# This code identifies the top three batsmen for the "Orange Cap" (highest run-scorer) in each season based on their total runs scored.

temp = new_df.groupby(['season','batsmanName'])['runs'].sum()
ranked = temp.groupby('season').rank(ascending=False)

orange_cap = ranked.nsmallest(3)

In [667]:
orange_cap

season  batsmanName   
 2021   RuturajGaikwad    1.0
 2022   JosButtler        1.0
 2023   ShubmanGill       1.0
Name: runs, dtype: float64

In [668]:
# This code identifies the top three bowlers for the "Purple Cap" (highest wicket-taker) in each season based on their total wickets taken.

temp = bowling_data.groupby(['season','bowlerName'])['wickets'].sum()
ranked = temp.groupby('season').rank(ascending=False)
purple_cap = ranked.nsmallest(3)

In [669]:
purple_cap

season  bowlerName     
 2021   HarshalPatel       1.0
 2022   YuzvendraChahal    1.0
 2023   MohammedShami      1.0
Name: wickets, dtype: float64